In [5]:
import psycopg2 as ps
import pandas as pd

In [83]:
#Connecting to AWS RDS Database
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
        
    except ps.OperationalError as e:
        raise e
    
    else:
        print('Connected!')
    return conn

In [84]:
# Database Credentials

host_name = '*********'
port = '5432'
dbname= 'youtube_api_db'
username = '******'
password = '******'
conn = None

#Connecting to the AWS RDS
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [85]:
#Creating a table in the DB

def create_table(curr):
    create_table = ("""CREATE TABLE IF NOT EXISTS datasciencevideos(
            video_id VARCHAR(255) PRIMARY KEY,
            video_title TEXT NOT NULL,
            video_description TEXT NOT NULL,
            channel_title TEXT NOT NULL,
            date_upload DATE NOT NULL DEFAULT CURRENT_DATE,
            view_count INTEGER NOT NULL,
            like_count INTEGER NOT NULL,
            favorite_count INTEGER NOT NULL,
            comment_count INTEGER NOT NULL
    )""")
    
    curr.execute(create_table)

In [86]:
#checking if video exists
def check_vid_exists(curr, video_id):
    query = ("""SELECT video_id FROM datasciencevideos WHERE video_id = %s""")
    curr.execute(query, (video_id,))
    
    return curr.fetchone() is not None

In [96]:
#update row if video exists

def update_row(curr, video_id, video_title, video_description, channel_title, date_upload, view_count,like_count, 
              favorite_count, comment_count):
    query = ("""UPDATE datasciencevideos
                SET video_id = %s,
                    video_title = %s,
                    video_description = %s,
                    channel_title = %s,
                    date_upload = %s,
                    view_count = %s,
                    like_count = %s,
                    favorite_count = %s,
                    comment_count = %s
                WHERE video_id = %s;""")
                    

    to_update = (video_id,video_title, video_description, channel_title, date_upload, view_count,like_count,
                 favorite_count,comment_count)
    curr.execute(query, to_update)

In [97]:
#updating the database
#handling data for scalability

#check to see if video exists

def update_db(curr,df):
    temp_df = pd.DataFrame(columns=['video_id', 'video_title', 'video_description', 'channel_title', 'date_upload',
                                    'view_count', 'like_count', 'favorite_count', 'comment_count'])
    for i, row in df.iterrows():
        if check_vid_exists(curr, row['video_id']): #if video exists, we'll update
            update_row(curr, row['video_id'], row['video_title'], row['video_description'], row['channel_title'], 
                       row['date_upload'], row['view_count'], row['like_count'], row['favorite_count'], 
                       row['comment_count'])
        else: #If video doesn't exists, append to db table
            temp_df = temp_df.append(row)

    return temp_df

In [98]:
#Writing an insert command

def insert_into_table(curr, video_id, video_title, video_description, channel_title, date_upload, view_count,like_count,
                      favorite_count, comment_count):
    insert_into_vid = ("""INSERT INTO datasciencevideos (video_id, video_title, video_description, channel_title, date_upload, view_count, 
                        like_count, favorite_count, comment_count)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s);""")
    row_to_insert = (video_id, video_title, video_description, channel_title, date_upload, view_count,like_count,
                      favorite_count, comment_count)
    curr.execute(insert_into_vid, row_to_insert)  

In [99]:
#Appending df data into Postgres DB. 
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr,row['video_id'], row['video_title'], row['video_description'], row['channel_title'], 
                       row['date_upload'], row['view_count'], row['like_count'], row['favorite_count'], 
                       row['comment_count'])
    

In [100]:
df = pd.read_csv('youtube_API_data.csv')
df.head()

,video_id,video_title,video_description,channel_title,date_upload,view_count,like_count,favorite_count,comment_count
0,xC-c7E5PK0Y,What REALLY is Data Science? Told by a Data Sc...,MERCH IS OUT FOR PRE-SALE: https://shopdoggos....,Joma Tech,2018-06-23,3347359,129564,0,3610
1,X3paOmcrTjQ,Data Science In 5 Minutes | Data Science For B...,Advanced Certificate Program In Data Science: ...,Simplilearn,2018-12-04,3312461,48545,0,1085
2,-ETQ97mXXF0,Data Science Full Course - Learn Data Science ...,"Data Science Master Program (Use Code ""YOUTUBE...",edureka!,2019-08-18,3073900,64512,0,683
3,FvGRKQipTP8,See Part 2 with the Data Scientist in the comm...,Watch Part 2 here: https://www.youtube.com/wat...,The Final Round,2022-07-11,2910837,90721,0,1368
4,LHBE6Q9XlzI,Python for Data Science - Course for Beginners...,This Python data science course will take you ...,freeCodeCamp.org,2020-06-02,2794386,67824,0,1284


In [101]:
curr = conn.cursor()

In [93]:
#creates table
create_table(curr)

In [ ]:
# Updates data for existing videos
new_vid_df = update_db(curr, df)

In [ ]:
#Inserting new videos into the Database
append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [ ]:
#To view the data in the Postgres DB
curr.execute("SELECT * FROM datasciencevideos")
print(curr.fetchall())